<a href="https://colab.research.google.com/github/MariaAsghar/GradientDescentBackpropagation/blob/master/MLAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Implementation of Gradient Algorithm

In [4]:
import numpy as np

file_path = '/content/mnistset.bin'  # Replace with the correct file path
# Load the binary file into a NumPy array
training_data = np.fromfile(file_path, dtype=np.float32)
# Print the array
print("training_data")
print(training_data)

file_path = '/content/labels.bin'  # Replace with the correct file path
# Load the binary file into a NumPy array
training_labels = np.fromfile(file_path, dtype=np.int32)
# Print the array
print("training_labels")
print(training_labels)

file_path = '/content/mnistset_test.bin'  # Replace with the correct file path
# Load the binary file into a NumPy array
test_data = np.fromfile(file_path, dtype=np.float32)
# Print the array
print("test_data")
print(test_data)

file_path = '/content/labels.bin'  # Replace with the correct file path
# Load the binary file into a NumPy array
test_labels = np.fromfile(file_path, dtype=np.float32)
# Print the array
print("test_labels")
print(test_labels)

training_data
[2.2366853e+08 1.2387478e-40 3.4833396e-15 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00]
training_labels
[1297436307      88400  662372422 ...          0          8          0]
test_data
[2.2366853e+08 1.2387478e-40 3.4833497e-15 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00]
test_labels
[2.2366853e+08 1.2387478e-40 3.4833396e-15 ... 0.0000000e+00 1.1210388e-44
 0.0000000e+00]


Main Py

In [5]:
import numpy as np

#set the data and labels from the training and test sets
#the data must have form list of lists such as [example 1, example 2, ...]
#where each example is a list of features, such as example 1 = [pixel 1, pixel 2, ...]

norm = 1.0 #normalize data
check_test = True #check the performance on the testing data after each epoch
number_examples_test = 100 #number of examples in the test set
hot_label = True#true if the label need to be transformed to one-hot enconding scheme, false if the label is the output layer target. Set False for regression problems.

#set network // format: [input layer, hidden layer 1, hidden layer 2, ..., output layer]
net = [784,100,10]

#set training parameters
epochs = 30 #number of times the network will train with the training set
number_examples_training = 100 #number of examples provided to network from the training set
mini_batch_size = 1 #number of examples to calculate the gradients before update the learning parameters
suffle_training = True #shuffle the examples in the training set

#set hyperparameters
learning_rate = 0.01 #size of the gradient to update the learning parameters
momentum_rate = 0.0 #rate of the momentum for optimization, 0 if SGD
tau = 100.0 #time step size
period = 1 #time steps

######################################################################################################################
def data():
    return [norm,training_data,training_labels,test_data,test_labels,check_test,number_examples_test,hot_label]

def network():
    return [net, epochs, number_examples_training,mini_batch_size,suffle_training,learning_rate,momentum_rate,tau,period]


Test py

In [6]:
import numpy as np
import  pickle

#load the data from the main script
norm = data()[0]
test_data = data()[3]
test_labels = data()[4]
total = data()[6]
hot_label = data()[7]

#load the parameters from the main script
net = network()[0]

#activation function
def f(z):
    if z>100.0:
        return 1.0
    elif z<-100.0:
        return 0.0
    else:
        return 1.0/(1.0+np.exp(-z))

x=[] #input to each neuron
y=[] #activation of each neuron
for layer in range(len(net)):
    x.append(np.array([0.0]*net[layer]))
    y.append(np.array([0.0]*net[layer]))

def getScore():
    score_test = 0.0
    weights = pickle.load( open( "weights", "rb" ) )
    loss_test = 0.0
    
    for example in range(total):
        #update activation of each neuron
        y[0] = test_data[example]/norm
        for layer in range(1,len(net)):
            x[layer]=np.dot(weights[layer-1],y[layer-1])
            y[layer]=map(f,x[layer])
        
        #guess the class from classifcation problem
        guess = np.argmax(y[len(net)-1])
        
        if hot_label==True:
            target = np.array([0.0]*net[len(net)-1])
            target[test_labels[example]] = 1.0
        else:
            target = test_labels[example]
        
        #classification performance on the test data
        if hot_label == True: #for classification problems
            if guess == test_labels[example]:
                score_test += 1.0
        loss_test += (0.5/total)*(1.0/net[len(net)-1])*sum( (target - y[len(net)-1])**2.0)
    
    if hot_label == True:
        return score_test/total, loss_test
    else:
        return loss_test


Target py

In [10]:
#-----------------------------------------------Atefeh-------
import random, sys ,pickle
import numpy as np
import  test

#load the data from the main script
norm = data()[0]
training_data = data()[1]
training_labels = data()[2]
check_test = data()[5]
hot_label = data()[7]

#load the parameters from the main script
net = network()[0]
epochs = network()[1]
number_examples = network()[2]
mini_batch_size = network()[3]
suffle_training = network()[4]
learning_rate = network()[5]
momentum_rate = network()[6]
tau = network()[7]
period = network()[8]

#activation function
def f(z):
    if z>100.0:
        return 1.0
    elif z<-100.0:
        return 0.0
    else:
        return 1.0/(1.0+np.exp(-z))

#derivative of the activation function
def fprime(z):
    return f(z)*(1.0-f(z))

x=[] #input to each neuron
y=[] #activation of each neuron
target=[] #target of each neuron
for layer in range(len(net)):
    x.append(np.array([0.0]*net[layer]))
    y.append(np.array([0.0]*net[layer]))
    target.append(np.array([0.0]*net[layer]))

#initialize weights and momentum
weights=[]
momentum=[]
for layer in range(len(net)-1):
    if layer==0:
        sigma=(48.0/(35.0*net[layer]))**0.5
    else:
        sigma=(16.0/(11.0*net[layer]))**0.5
    momentum.append(np.zeros((net[layer+1],net[layer])))
    weights.append(np.random.normal(0.0,sigma,(net[layer+1],net[layer])))

##-------------------------------------------------Maria
#initialize gradients
gradients=[]
for layer in range(len(net)-1):
    gradients.append(np.zeros((net[layer+1],net[layer])))

list_of_examples = []
for example in range(number_examples):
    list_of_examples.append(example)

for trials in range(epochs):
    score_training = 0.0
    example_counter = 0.0
    batch_counter = 0
    score_training = 0.0
    loss_training = 0.0
    
    if suffle_training == True:
        random.shuffle(list_of_examples)
    
    for example in list_of_examples:
        example_counter += 1.0
        batch_counter += 1
        
        if hot_label==True:
            print("hello1")
            target[len(net)-1] = np.array([0.0]*net[len(net)-1])
            print("hello2")
            target[len(net)-1][training_labels[example]] = 1.0
            ##
        else:
            target[len(net)-1] = training_labels[example]

        #update activation of each neuron
        y[0] = training_data[example]/norm
        for layer in range(1,len(net)):
            print(type(weights[layer-1]))
            print(type(y[layer-1]))
            x[layer]=np.dot(weights[layer-1],y[layer-1])
            y[layer]=map(f,x[layer])
        
        #guess the class from classifcation problem
        if hot_label == True:
            guess = np.argmax(y[len(net)-1])
           
        #calculate online loss on training data
        loss_training += (0.5/number_examples)*(1.0/net[len(net)-1])*sum( (target[len(net)-1] - y[len(net)-1])**2.0)
#---------------------------Shahzeb--------------------------------------      
        if batch_counter <= mini_batch_size:
            #compute gradient from the output layer
            gradient_output = (y[len(net)-1]-target[len(net)-1])*map(fprime,x[len(net)-1])
            gradients[len(net)-2] += np.outer(gradient_output,y[len(net)-2])
            
            graph = open('score','w', 0)
            #compute targets for all hidden layers
            
            for layer in range(len(net)-2,0,-1):
                target[layer] = np.array([0.0]*net[layer])
                x_hat = x[layer+1][:]
                y_hat = y[layer+1][:]
                
                for time_steps in range(period):
                    target[layer] += -tau*np.dot(np.transpose(weights[layer]),(y_hat-target[layer+1])*map(fprime,x_hat))
                    x_hat = np.dot(weights[layer],target[layer])
                    y_hat = map(f,x_hat)

                #compute gradients from the hidden layers
                gradients[layer-1] += -np.outer((target[layer]-y[layer])*map(fprime,x[layer]),y[layer-1])
                
        #update the learning parameters
        if batch_counter == mini_batch_size:
            for layer in range(0,len(net)-1):
                momentum[layer] = momentum_rate*momentum[layer] - (1.0-momentum_rate)*learning_rate*gradients[layer]/mini_batch_size
                weights[layer] += momentum[layer]
  #------------------------------Nawordth---------------------------          
            #reset batch counter and gradients
            batch_counter = 0
            for layer in range(len(net)-1):
                gradients[layer] = np.zeros((net[layer+1],net[layer]))
            
        #online classification performance on the training data
        if hot_label == True:
            if guess == training_labels[example]:
                score_training += 1.0
            
        sys.stdout.write("\r%f %f %i %i" % (score_training/example_counter,loss_training,example_counter,trials))
        sys.stdout.flush()
        
    #save the weights after each epoch
    address = file("weights","wb")
    pickle.dump(weights, address)
    address.close()
    
    if check_test == True:
        score_test = test.getScore()
        print (' ')
        print('epoch: ',trials, ' score test, loss: ',score_test)


hello1
hello2
<class 'numpy.ndarray'>
<class 'numpy.float64'>
<class 'numpy.ndarray'>
<class 'map'>


TypeError: ignored

In [ ]:
     for example in list_of_examples:
        example_counter += 1.0
        batch_counter += 1
        
        if hot_label==True:
            target[len(net)-1] = np.array([0.0]*net[len(net)-1])
            target[len(net)-1][training_labels[example]] = 1.0
        else:
            target[len(net)-1] = training_labels[example]


IndexError: ignored

In [ ]:
training_labels[example]

3.4833396e-15

In [ ]:
import random, sys ,pickle
import numpy as np
import  test

#load the data from the main script
norm = data()[0]
training_data = data()[1]
training_labels = data()[2]
check_test = data()[5]
hot_label = data()[7]

#load the parameters from the main script
net = network()[0]
epochs = network()[1]
number_examples = network()[2]
mini_batch_size = network()[3]
suffle_training = network()[4]
learning_rate = network()[5]
momentum_rate = network()[6]
tau = network()[7]
period = network()[8]

#activation function
def f(z):
    if z>100.0:
        return 1.0
    elif z<-100.0:
        return 0.0
    else:
        return 1.0/(1.0+np.exp(-z))

#derivative of the activation function
def fprime(z):
    return f(z)*(1.0-f(z))

x=[] #input to each neuron
y=[] #activation of each neuron
target=[] #target of each neuron
for layer in range(len(net)):
    x.append(np.array([0.0]*net[layer]))
    y.append(np.array([0.0]*net[layer]))
    target.append(np.array([0.0]*net[layer]))

#initialize weights and momentum
weights=[]
momentum=[]
for layer in range(len(net)-1):
    if layer==0:
        sigma=(48.0/(35.0*net[layer]))**0.5
    else:
        sigma=(16.0/(11.0*net[layer]))**0.5
    momentum.append(np.zeros((net[layer+1],net[layer])))
    weights.append(np.random.normal(0.0,sigma,(net[layer+1],net[layer])))

#initialize gradients
gradients=[]
for layer in range(len(net)-1):
    gradients.append(np.zeros((net[layer+1],net[layer])))

list_of_examples = []
for example in range(number_examples):
    list_of_examples.append(example)

for trials in range(epochs):
    score_training = 0.0
    example_counter = 0.0
    batch_counter = 0
    score_training = 0.0
    loss_training = 0.0
    
    if suffle_training == True:
        random.shuffle(list_of_examples)
    
    for example in list_of_examples:
        example_counter += 1.0
        batch_counter += 1
        
        if hot_label==True:
            target[len(net)-1] = np.array([0.0]*net[len(net)-1])
            target[len(net)-1][training_labels[example]] = 1.0
        else:
            target[len(net)-1] = training_labels[example]
        
        #update activation of each neuron
        y[0] = training_data[example]/norm
        for layer in range(1,len(net)):
            x[layer]=np.dot(weights[layer-1],y[layer-1])
            y[layer]=map(f,x[layer])
        
        #guess the class from classifcation problem
        if hot_label == True:
            guess = np.argmax(y[len(net)-1])
        
        #calculate online loss on training data
        loss_training += (0.5/number_examples)*(1.0/net[len(net)-1])*sum( (target[len(net)-1] - y[len(net)-1])**2.0)
        
        if batch_counter <= mini_batch_size:
            #compute gradient from the output layer
            gradient_output = (y[len(net)-1]-target[len(net)-1])*map(fprime,x[len(net)-1])
            gradients[len(net)-2] += np.outer(gradient_output,y[len(net)-2])
            
            graph = open('score','w', 0)
            #compute targets for all hidden layers
            
            for layer in range(len(net)-2,0,-1):
                target[layer] = np.array([0.0]*net[layer])
                x_hat = x[layer+1][:]
                y_hat = y[layer+1][:]
                
                for time_steps in range(period):
                    target[layer] += -tau*np.dot(np.transpose(weights[layer]),(y_hat-target[layer+1])*map(fprime,x_hat))
                    x_hat = np.dot(weights[layer],target[layer])
                    y_hat = map(f,x_hat)

                #compute gradients from the hidden layers
                gradients[layer-1] += -np.outer((target[layer]-y[layer])*map(fprime,x[layer]),y[layer-1])
                
        #update the learning parameters
        if batch_counter == mini_batch_size:
            for layer in range(0,len(net)-1):
                momentum[layer] = momentum_rate*momentum[layer] - (1.0-momentum_rate)*learning_rate*gradients[layer]/mini_batch_size
                weights[layer] += momentum[layer]
            
            #reset batch counter and gradients
            batch_counter = 0
            for layer in range(len(net)-1):
                gradients[layer] = np.zeros((net[layer+1],net[layer]))
            
        #online classification performance on the training data
        if hot_label == True:
            if guess == training_labels[example]:
                score_training += 1.0
            
        sys.stdout.write("\r%f %f %i %i" % (score_training/example_counter,loss_training,example_counter,trials))
        sys.stdout.flush()
        
    #save the weights after each epoch
    address = file("weights","wb")
    pickle.dump(weights, address)
    address.close()
    
    if check_test == True:
        score_test = test.getScore()
        print (' ')
        print('epoch: ',trials, ' score test, loss: ',score_test)


IndexError: ignored